In [ ]:
from utils import get_stock_price_data, LIST_STOCK
from sklearn.linear_model import LinearRegression
import numpy as np
from tqdm import tqdm

In [ ]:
time_span = 7
future_cols = [f'close_t{i}' for i in range(0, time_span)]
X = np.array(range(0, len(future_cols))).reshape(-1, 1)
def compute_slope_future(row):
    y = row[future_cols].values.astype(float)
    model = LinearRegression()
    model.fit(X, y)
    return model.coef_[0]

In [ ]:
threshold = 0.03 / time_span # Tăng giảm 3% trong time_span
for symbol in tqdm(LIST_STOCK):
    df = get_stock_price_data(symbol)
    for i in range(0, time_span):
        df[f'close_t{i}'] = df['close'].shift(-i)
    df.dropna(inplace=True)
    for i in range(time_span-1, -1, -1):
        df[f'close_t{i}'] = df[f'close_t{i}']/df[f'close']

    coef_col_name = f'coef_close_t0_to_close_t{time_span-1}'
    df[coef_col_name] = df.apply(compute_slope_future, axis=1)
    df[f'ground_truth'] = np.where(
        df[coef_col_name] >= threshold, 'up',
        np.where(df[coef_col_name] <= -threshold, 'down', 'sideways')
    )
    df.to_csv(f'./data/data/{symbol}/history_price_with_gt.csv', index=False)